<a href="https://colab.research.google.com/github/DEFINER24/2025DCC/blob/main/Yolo%EB%AA%A8%EB%8D%B8%ED%8C%8C%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO 모델 파일
## 0. 구글 드라이브 마운트 및 압축 해제

In [ ]:
# ==========================================================
# 0. 구글 드라이브 마운트 및 압축 해제
# ==========================================================
from google.colab import drive
drive.mount('/content/drive')

import zipfile, os
from pathlib import Path

# ZIP 파일 경로 지정
zip_path = '/content/drive/MyDrive/model_data/Model_data.zip'
extract_path = '/content/drive/MyDrive/data_model'

# 압축 해제
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"압축 해제 완료: {extract_path}")
else:
    print("⚠️ Model_data.zip 파일을 찾을 수 없습니다.")


ModuleNotFoundError: No module named 'google'

In [ ]:
# 압축 해제 후 폴더 구조 점검
import os
for folder in ['train', 'val', 'test']:
    full_path = os.path.join('./Model_data', folder)
    if os.path.exists(full_path):
        n_img = len(os.listdir(os.path.join(full_path, 'images')))
        n_lbl = len(os.listdir(os.path.join(full_path, 'labels_json')))
        print(f"{folder}: images {n_img}개, labels {n_lbl}개")
    else:
        print(f"{folder} 폴더가 존재하지 않습니다.")


train: images 68376개, labels 68376개
val: images 8545개, labels 8545개
test: images 8545개, labels 8545개


## 본 코드

In [1]:
# ==========================================================
# 0. 환경 설정 및 라이브러리 설치
# ==========================================================
#!pip -q install ultralytics==8.3.50
import os, json, glob, random
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import pandas as pd
import numpy as np
import torch
from ultralytics import YOLO


In [2]:
print("Torch:", torch.__version__)
print("GPU available:", torch.cuda.is_available())

Torch: 2.5.1+cu121
GPU available: True


In [4]:
# ==========================================================
# 1. 구글 드라이브 마운트 및 데이터 경로 지정
# ==========================================================
#from google.colab import drive
#drive.mount('/content/drive')

# 실제 폴더 구조 반영
DATA_ROOT = Path('./Model_data').resolve()
assert (DATA_ROOT / 'train').exists(), "train 폴더 없음"
assert (DATA_ROOT / 'val').exists(), "val 폴더 없음"
assert (DATA_ROOT / 'test').exists(), "test 폴더 없음"


In [3]:
# ==========================================================
# 2. 클래스 정의
# ==========================================================
CLASS_NAMES = ['pipe', 'box', 'manhole', 'patch', 'cavity']
CLASS_TO_ID = {c: i for i, c in enumerate(CLASS_NAMES)}
print("CLASS_TO_ID:", CLASS_TO_ID)


CLASS_TO_ID: {'pipe': 0, 'box': 1, 'manhole': 2, 'patch': 3, 'cavity': 4}


In [6]:
IMG_W, IMG_H = 300, 256      # 모든 GPR 이미지 해상도           # XZ 평면만 사용 (전부 사용하려면 None)

def parse_one_json(jpath: Path, img_dir: Path):
    with open(jpath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    plane = (data.get('plane') or '').strip().upper()

    img_name = data.get('fileName')
    img_path = img_dir / img_name
    # targetType으로 클래스 결정
    target = (data.get('targetType') or '').strip().lower()
    if target not in CLASS_TO_ID:
        return img_path, [], plane

    cid = CLASS_TO_ID[target]
    ann = data.get('annotation', {})
    anns = ann if isinstance(ann, list) else [ann]
    yolo_lines = []

    for a in anns:
        if not all(k in a for k in ('bbox_x', 'bbox_y', 'bbox_w', 'bbox_h')):
            continue
        bx, by = float(a['bbox_x']), float(a['bbox_y'])
        bw, bh = float(a['bbox_w']), float(a['bbox_h'])
        if a.get('bbox_xM') and a.get('bbox_yM'):
            try:
                bxM, byM = float(a['bbox_xM']), float(a['bbox_yM'])
                bw = max(1.0, bxM - bx)
                bh = max(1.0, byM - by)
            except Exception:
                pass

        # YOLO 좌표 정규화
        cx = (bx + bw/2.0) / IMG_W
        cy = (by + bh/2.0) / IMG_H
        nw = bw / IMG_W
        nh = bh / IMG_H
        cx = min(max(cx, 0.0), 1.0)
        cy = min(max(cy, 0.0), 1.0)
        nw = min(max(nw, 0.0), 1.0)
        nh = min(max(nh, 0.0), 1.0)

        yolo_lines.append(f"{cid} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")

    return img_path, yolo_lines, plane


In [8]:
# ==========================================================
# 4. YZ 평면 데이터만 변환
# ==========================================================
def convert_split(split_name: str):
    split_dir = DATA_ROOT / split_name
    img_dir   = split_dir / 'images'
    json_dir  = split_dir / 'labels_json'        # 원본 JSON
    yolo_lbl  = split_dir / 'labels'   # 변환 결과 저장
    yolo_lbl.mkdir(exist_ok=True)

    json_files = sorted(glob.glob(str(json_dir / '*.json')))
    ok, skip, noimg, noline = 0, 0, 0, 0
    per_class_counts = {k: 0 for k in CLASS_NAMES}

    for jp in json_files:
        jp = Path(jp)
        img_path, yolo_lines, plane = parse_one_json(jp, img_dir)

        if img_path is None and yolo_lines is None:
            skip += 1
            continue
        if img_path is None:
            noimg += 1
            continue
        if not yolo_lines:
            noline += 1
            continue

        out_lbl = yolo_lbl / f"{img_path.stem}.txt"
        with open(out_lbl, 'w', encoding='utf-8') as f:
            f.write('\n'.join(yolo_lines))

        cid = int(yolo_lines[0].split()[0])
        per_class_counts[CLASS_NAMES[cid]] += 1
        ok += 1

    print(f"[{split_name}] 변환 성공: {ok}, 제외: {skip}, 이미지없음: {noimg}, 라벨없음: {noline}")
    print(f"[{split_name}] 클래스 분포:", per_class_counts)
    return ok

convert_split('train')
convert_split('val')
convert_split('test')

[train] 변환 성공: 68376, 제외: 0, 이미지없음: 0, 라벨없음: 0
[train] 클래스 분포: {'pipe': 27467, 'box': 8737, 'manhole': 10395, 'patch': 9607, 'cavity': 12170}
[val] 변환 성공: 8545, 제외: 0, 이미지없음: 0, 라벨없음: 0
[val] 클래스 분포: {'pipe': 3433, 'box': 1092, 'manhole': 1299, 'patch': 1200, 'cavity': 1521}
[test] 변환 성공: 8545, 제외: 0, 이미지없음: 0, 라벨없음: 0
[test] 클래스 분포: {'pipe': 3433, 'box': 1092, 'manhole': 1299, 'patch': 1200, 'cavity': 1521}


8545

In [9]:
# ==========================================================
# 5. data.yaml 생성
# ==========================================================
YAML_PATH = DATA_ROOT / 'data.yaml'
yaml_text = f"""
path: {DATA_ROOT.as_posix()}
train: train/images
val: val/images
test: test/images
names:
  0: pipe
  1: box
  2: manhole
  3: patch
  4: cavity
"""
with open(YAML_PATH, 'w', encoding='utf-8') as f:
    f.write(yaml_text)
print(f"✅ data.yaml 생성 완료: {YAML_PATH}")



✅ data.yaml 생성 완료: C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\data.yaml


In [10]:
# ==========================================================
# 6. YOLO 모델 학습 (yz 평면 데이터만)
# ==========================================================
MODEL_NAME = 'yolo11m.pt'
IMG_SIZE   = 320
EPOCHS     = 5
#EPOCHS     = 100 --- IGNORE ---
BATCH      = -1

model = YOLO(MODEL_NAME)
results = model.train(
    data=str(YAML_PATH),
    imgsz=IMG_SIZE,
    epochs=EPOCHS,
    batch=BATCH,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=str(DATA_ROOT / 'runs'),
    name='yolo_model',
    verbose=True
)

best_ckpt = Path(results.save_dir) / 'weights' / 'best.pt'
print("최적 가중치 경로:", best_ckpt)


New https://pypi.org/project/ultralytics/8.3.214 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.50  Python-3.12.10 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolo11m.pt, data=C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\data.yaml, epochs=5, time=None, patience=100, batch=-1, imgsz=320, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\runs, name=yolo_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fa

train: Scanning C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\train\labels.cache... 68376 images, 0 backgrounds, 0 corrupt: 100%|██████████| 68376/68376 [00:00<?, ?it/s]


AutoBatch: Computing optimal batch size for imgsz=320 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU) 8.00G total, 0.19G reserved, 0.18G allocated, 7.62G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    20056863       17.05         0.415         99.15           252        (1, 3, 320, 320)                    list
    20056863        34.1         0.514         97.02         108.4        (2, 3, 320, 320)                    list
    20056863       68.21         0.763          51.5         63.33        (4, 3, 320, 320)                    list
    20056863       136.4         1.294         54.16         74.42        (8, 3, 320, 320)                    list
    20056863       272.8         2.290         65.04          79.5       (16, 3, 320, 320)                    list
AutoBatch: Using batch-size 34 for CUDA:0 4.94G/8.00G (62%) 


train: Scanning C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\train\labels.cache... 68376 images, 0 backgrounds, 0 corrupt: 100%|██████████| 68376/68376 [00:00<?, ?it/s]
val: Scanning C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\val\labels.cache... 8545 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8545/8545 [00:00<?, ?it/s]


Plotting labels to C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\runs\yolo_model\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.00053125), 112 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 8 dataloader workers
Logging results to C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\runs\yolo_model
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        5/5       4.9G     0.3919      1.186      1.009          3        320: 100%|██████████| 2012/2012 [09:52<00:00,  3.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 126/126 [00:51<00:00,  2.43it/s]

                   all       8545       8545      0.513      0.859       0.59      0.555



5 epochs completed in 0.902 hours.
Optimizer stripped from C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\runs\yolo_model\weights\last.pt, 40.5MB
Optimizer stripped from C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\runs\yolo_model\weights\best.pt, 40.5MB

Validating C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\runs\yolo_model\weights\best.pt...
Ultralytics 8.3.50  Python-3.12.10 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11m summary (fused): 303 layers, 20,033,887 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 126/126 [00:59<00:00,  2.11it/s]


                   all       8545       8545      0.513      0.859       0.59      0.555
                  pipe       3433       3433       0.48      0.923      0.519      0.498
                   box       1092       1092      0.476      0.826      0.487      0.449
               manhole       1299       1299      0.658      0.965       0.91      0.862
                 patch       1200       1200      0.469      0.764      0.506       0.45
                cavity       1521       1521      0.481      0.819      0.529      0.517
Speed: 0.1ms preprocess, 1.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\runs\yolo_model
최적 가중치 경로: C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\runs\yolo_model\weights\best.pt


In [18]:
best_model = YOLO(best_ckpt)
val_res = best_model.val(
    data=str(YAML_PATH),
    split='test',
    imgsz=IMG_SIZE,
    device=0 if torch.cuda.is_available() else 'cpu',
    conf=0.001,
    iou=0.5,
    max_det=3000
)

stats = val_res.results_dict
mAP50 = float(stats['metrics/mAP50(B)'])
print(f"mAP@IoU=0.5: {mAP50:.4f}")



Ultralytics 8.3.50  Python-3.12.10 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11m summary (fused): 303 layers, 20,033,887 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning C:\Users\k0107\Documents\GitHub\2025DCC\Model_data\test\labels.cache... 8545 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8545/8545 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 535/535 [01:06<00:00,  8.10it/s]


                   all       8545       8545      0.519      0.856      0.599       0.56
                  pipe       3433       3433      0.483      0.917      0.524      0.502
                   box       1092       1092      0.482      0.829      0.495      0.456
               manhole       1299       1299       0.68       0.96      0.913       0.86
                 patch       1200       1200      0.471      0.737      0.515      0.444
                cavity       1521       1521      0.481      0.837       0.55       0.54
Speed: 0.1ms preprocess, 3.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to c:\Users\k0107\Documents\GitHub\2025DCC\runs\detect\val5
mAP@IoU=0.5: 0.5994
